In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import datetime

import json
import math
import os
from pathlib import Path
from PIL import Image
import gc
import time
import struct

from typing import List, NamedTuple, Optional, Tuple

import cv2
import imageio

import numpy as np

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import roma
# import torch_levenberg_marquardt as tlm
import matplotlib.pyplot as plt

import sys
sys.path.append(os.path.abspath(os.path.join("external", "bop_toolkit")))
from bop_toolkit_lib import inout
sys.path.append(os.path.abspath(os.path.join("external", "dinov2")))

from utils.misc import array_to_tensor, tensor_to_array, tensors_to_arrays


from utils import (
    corresp_util,
    config_util,
    eval_errors,
    eval_util,
    feature_util,
    infer_pose_util,
    knn_util,
    misc as misc_util,
    pnp_util,
    projector_util,
    repre_util,
    vis_util,
    data_util,
    renderer_builder,
    json_util, 
    logging,
    misc,
    structs,
    geometry
)

from utils.structs import AlignedBox2f, PinholePlaneCameraModel, CameraModel
from utils.misc import warp_depth_image, warp_image

In [ ]:
extractor_name = "dinov2_version=vitl14-reg_stride=14_facet=token_layer=18_logbin=0_norm=1"
output_path = Path("/scratch/jeyan/foundpose/output_barrelddt1_raw_vitl_layer18")
dataset_path = Path("/scratch/jeyan/barreldata/divedata/dive8/barrelddt1/rgb")
mask_path = Path("/scratch/jeyan/barreldata/results/barrelddt1/masks")
cam_json_path = Path("/scratch/jeyan/barreldata/divedata/dive8/barrelddt1/camera.json")
crop_size = (420, 420)
grid_cell_size = 14.0
crop_rel_pad = 0.2

imgpaths = sorted(list(dataset_path.glob("*.jpg")) + list(dataset_path.glob("*.png")))
maskpaths = sorted(list(mask_path.glob("*.png")))

# Prepare feature extractor.
extractor = feature_util.make_feature_extractor(extractor_name)
# Prepare a device.
device = "cuda" if torch.cuda.is_available() else "cpu"
extractor.to(device)

# Create a pose evaluator.
pose_evaluator = eval_util.EvaluatorPose([0])

# Load the object representation.
# logger.info(
#     f"Loading representation for object {0} from dataset {opts.object_dataset}..."
# )
base_repre_dir = Path(output_path, "object_repre")
repre_dir = base_repre_dir
repre: repre_util.FeatureBasedObjectRepre = repre_util.load_object_repre(
    repre_dir=repre_dir,
    tensor_device=device,
)

repre_np = repre_util.convert_object_repre_to_numpy(repre)

In [ ]:
repre_np.feat_vectors.shape

In [ ]:
feattempmask = repre_np.feat_to_template_ids == 0
# all p_i values
repre_np.feat_vectors[feattempmask].shape
# all x_i values
repre_np.vertices[repre_np.feat_to_vertex_ids[feattempmask]].shape

In [ ]:
repre_np.vertices[repre_np.feat_to_vertex_ids[feattempmask]].shape

In [ ]:
i = 0
imgpath = imgpaths[i]

In [ ]:
# Camera parameters.
# transform is from GT, can we just leave as identity?
with open(Path(cam_json_path), "r") as f:
    camjson = json.load(f)
orig_camera_c2w = PinholePlaneCameraModel(
    camjson["width"], camjson["height"],
    (camjson["fx"], camjson["fy"]), (camjson["cx"], camjson["cy"])
)
orig_image_size = (
    orig_camera_c2w.width,
    orig_camera_c2w.height,
)

# Generate grid points at which to sample the feature vectors.
grid_size = crop_size
grid_points = feature_util.generate_grid_points(
    grid_size=grid_size,
    cell_size=grid_cell_size,
)
grid_points = grid_points.to(device)


# Estimate pose for each object instance.
times = {}

# Get the input image.
orig_image_np_hwc = np.array(Image.open(imgpath)) / 255.0

# Get the modal mask and amodal bounding box of the instance.
# binary mask
orig_mask_modal = np.array(Image.open(maskpaths[i]).convert("L")) / 255.0
sumvert = np.sum(orig_mask_modal, axis=0)
left = np.where(sumvert > 0)[0][0]
right = np.where(sumvert > 0)[0][-1]
sumhor = np.sum(orig_mask_modal, axis=1)
bottom = np.where(sumhor > 0)[0][0]
top = np.where(sumhor > 0)[0][-1]
# bounding box of mask
orig_box_amodal = AlignedBox2f(
    left=left,
    top=top,
    right=right,
    bottom=bottom,
)

# Get box for cropping.
crop_box = misc_util.calc_crop_box(
    box=orig_box_amodal,
    make_square=True,
)

# Construct a virtual camera focused on the crop.
crop_camera_model_c2w = misc_util.construct_crop_camera(
    box=crop_box,
    camera_model_c2w=orig_camera_c2w,
    viewport_size=crop_size,
    viewport_rel_pad=crop_rel_pad,
)

# Map images to the virtual camera.
interpolation = (
    cv2.INTER_AREA
    if crop_box.width >= crop_camera_model_c2w.width
    else cv2.INTER_LINEAR
)
image_np_hwc = warp_image(
    src_camera=orig_camera_c2w,
    dst_camera=crop_camera_model_c2w,
    src_image=orig_image_np_hwc,
    interpolation=interpolation,
)
mask_modal = warp_image(
    src_camera=orig_camera_c2w,
    dst_camera=crop_camera_model_c2w,
    src_image=orig_mask_modal,
    interpolation=cv2.INTER_NEAREST,
)

# Recalculate the object bounding box (it changed if we constructed the virtual camera).
ys, xs = mask_modal.nonzero()
box = np.array(misc_util.calc_2d_box(xs, ys))
box_amodal = AlignedBox2f(
    left=box[0],
    top=box[1],
    right=box[2],
    bottom=box[3],
)

# The virtual camera is becoming the main camera.
camera_c2w = crop_camera_model_c2w

# Extract feature map from the crop.
image_tensor_chw = array_to_tensor(image_np_hwc).to(torch.float32).permute(2, 0, 1).to(device)
image_tensor_bchw = image_tensor_chw.unsqueeze(0)
# BxDxHxW
extractor_output = extractor(image_tensor_bchw)
feature_map_chw = extractor_output["feature_maps"][0]

# Keep only points inside the object mask.
mask_modal_tensor = array_to_tensor(mask_modal).to(device)
query_points = feature_util.filter_points_by_mask(
    grid_points, mask_modal_tensor
)

# Extract features at the selected points, of shape (num_points, feat_dims).
query_features = feature_util.sample_feature_map_at_points(
    feature_map_chw=feature_map_chw,
    points=query_points,
    image_size=(image_np_hwc.shape[1], image_np_hwc.shape[0]),
).contiguous()

# Potentially project features to a PCA space.
if (
    query_features.shape[1] != repre.feat_vectors.shape[1]
    and len(repre.feat_raw_projectors) != 0
):
    query_features_proj = projector_util.project_features(
        feat_vectors=query_features,
        projectors=repre.feat_raw_projectors,
    ).contiguous()

    _c, _h, _w = feature_map_chw.shape
    feature_map_chw_proj = (
        projector_util.project_features(
            feat_vectors=feature_map_chw.permute(1, 2, 0).view(-1, _c),
            projectors=repre.feat_raw_projectors,
        )
        .view(_h, _w, -1)
        .permute(2, 0, 1)
    )
else:
    query_features_proj = query_features
    feature_map_chw_proj = feature_map_chw

In [ ]:
feature_map_chw.shape

In [ ]:
query_features_proj.shape, feature_map_chw_proj.shape

In [ ]:
R = np.array([
    [0.9824963517866108, -0.18626298081482784, -0.0026496611057800646],
    [0.07184429208626533, 0.36576270685794776, 0.9279310534552504],
    [-0.1718700567889119, -0.9118792377557032, 0.37274245710604814]
])
t = np.array([0.027086201383687152, 0.21964677626568319, 5.018856479067855])

In [ ]:
template_id = 245
feattempmask = repre.feat_to_template_ids == template_id
# all p_i values
allpi = repre.feat_vectors[feattempmask]
# all x_i values
allxi = repre.vertices[feattempmask]
Fq = feature_map_chw_proj
templatecam = repre.template_cameras_cam_from_model[template_id]

In [ ]:
def robustlosstorch(x: torch.Tensor, a=-5, c=0.5):
    """
    Args:
        x (torch.Tensor): Nxd tensor
    
    Returns:
        loss (torch.Tensor): N tensor
    """
    return torch.sum((abs(a - 2) / a) * (((x / c) ** 2 / abs(a - 2) + 1) ** (a / 2) - 1), dim=-1)

def descriptor_from_pose(
    q: torch.Tensor, t: torch.Tensor, camera: CameraModel, patchvtxs: torch.Tensor,
    featmap: torch.Tensor, patchsize: int
):
    T = roma.RigidUnitQuat(q, t)
    camf = torch.tensor(camera.f).cuda()
    camc = torch.tensor(camera.c).cuda()
    projected = camera.project(T[None].apply(patchvtxs)) * camf + camc
    patchproj = projected / patchsize
    projfeatures = feature_util.sample_feature_map_at_points(featmap, patchproj, featmap.shape[1:])
    return projfeatures

def featuremetric_loss(
    q: torch.Tensor, t: torch.Tensor, camera: CameraModel, patchdescs: torch.Tensor,
    patchvtxs: torch.Tensor, featmap: torch.Tensor, patchsize: int, a: float=-5, c: float=0.5
):
    """
    Args:
        q: quaternion input
        t: translation input
    """
    # need: projected features from every patch in query rgb image
    # template descriptor + 3d point from patches inside mask
    # coarse R and t, need to parametrize R as quaternion or axangle
    # tfm is 6d [*axangle, *t]
    projfeatures = descriptor_from_pose(q, t, camera, patchvtxs, featmap, patchsize)
    featurediff = patchdescs - projfeatures
    return robustlosstorch(featurediff, a=a, c=c)

def featuremetric_cost(
    q: torch.Tensor, t: torch.Tensor, camera: CameraModel, patchdescs: torch.Tensor,
    patchvtxs: torch.Tensor, featmap: torch.Tensor, patchsize: int, a: float=-5, c: float=0.5
):
    """
    Args:
        q: quaternion input
        t: translation input
    """
    return torch.sum(featuremetric_loss(q, t, camera, patchdescs, patchvtxs, featmap, patchsize, a=a, c=c))

class PoseDescriptorModel(nn.Module):
    def __init__(self, q: torch.Tensor, t: torch.Tensor, camera: CameraModel, patchvtxs: torch.Tensor, featmap: torch.Tensor, patchsize: int):
        super().__init__()
        self.qtorch = roma.rotmat_to_unitquat(torch.tensor(R)).float().cuda().requires_grad_()
        self.ttorch = torch.tensor(t).float().cuda().requires_grad_()
        self.camera = camera
        self.featmap = featmap
        self.patchsize = patchsize

    def forward(self, patchvtxs: torch.Tensor):
        return descriptor_from_pose(self.qtorch, self.ttorch, self.camera, patchvtxs, self.featmap, self.patchsize)

# class RobustLoss(tlm.loss.Loss):
#     def forward(self, y_true: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor:
#         return torch.sum(robustlosstorch(y_true - y_pred))

#     def residuals(self, y_true: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor:
#         return robustlosstorch(y_true - y_pred)

In [ ]:
qtorch = roma.rotmat_to_unitquat(torch.tensor(R)).float().cuda().requires_grad_()
ttorch = torch.tensor(t).float().cuda().requires_grad_()
print(qtorch, ttorch)
optimizer = torch.optim.Adam([qtorch, ttorch], lr=0.01)

losses = []
for _ in range(100):
    loss = featuremetric_cost(qtorch, ttorch, camera_c2w, allpi, allxi, Fq, 14)
    losses.append(loss.cpu().detach().numpy())
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        qtorch = roma.quat_normalize(qtorch)

In [ ]:
qtorch, ttorch

In [ ]:
feature_map_chw.shape

In [ ]:
image_np_hwc.shape